In [214]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

In [215]:
#data read/create dfs
prices_df = pd.read_csv('raw-data/egg-prices.csv')
production_df = pd.read_csv('raw-data/egg-production.csv')

In [216]:
#eda for prices_df
print(prices_df.head())
print(prices_df.tail())
print(prices_df.shape)
print(prices_df.columns)
print(prices_df.dtypes)
print(prices_df.describe(include='all'))

   YEAR  LOCATION STATE ANSI ASD CODE COUNTY ANSI REFERENCE PERIOD COMMODITY  \
0  2016  US TOTAL                                              JAN      EGGS   
1  2016  US TOTAL                                              FEB      EGGS   
2  2016  US TOTAL                                              MAR      EGGS   
3  2016  US TOTAL                                              APR      EGGS   
4  2016  US TOTAL                                              MAY      EGGS   

   ALL UTILIZATION PRACTICES in $ / DOZEN  \
0                                   1.180   
1                                   1.080   
2                                   0.970   
3                                   0.680   
4                                   0.552   

   ALL UTILIZATION PRACTICES, TABLE in $ / DOZEN  
0                                          0.969  
1                                          0.850  
2                                          0.731  
3                                          0

In [217]:
#eda for production_df
print(production_df.head())
print(production_df.tail())
print(production_df.shape)
print(production_df.columns)
print(production_df.dtypes)
print(production_df.describe(include='all'))

  Program  Year Period  Week Ending Geo Level     State  State ANSI  \
0  SURVEY  2021    APR          NaN     STATE   ALABAMA         1.0   
1  SURVEY  2021    APR          NaN     STATE   ALABAMA         1.0   
2  SURVEY  2021    APR          NaN     STATE   ALABAMA         1.0   
3  SURVEY  2021    APR          NaN     STATE   ALABAMA         1.0   
4  SURVEY  2021    APR          NaN     STATE  ARKANSAS         5.0   

   Ag District  Ag District Code  County  ...  Zip Code  Region  \
0          NaN               NaN     NaN  ...       NaN     NaN   
1          NaN               NaN     NaN  ...       NaN     NaN   
2          NaN               NaN     NaN  ...       NaN     NaN   
3          NaN               NaN     NaN  ...       NaN     NaN   
4          NaN               NaN     NaN  ...       NaN     NaN   

   watershed_code  Watershed  Commodity  \
0               0        NaN       EGGS   
1               0        NaN       EGGS   
2               0        NaN       EGGS  

In [218]:
#examining nulls
print(prices_df.isnull().sum())
print(production_df.isnull().sum())

YEAR                                             0
LOCATION                                         0
STATE ANSI                                       0
ASD CODE                                         0
COUNTY ANSI                                      0
REFERENCE PERIOD                                 0
COMMODITY                                        0
ALL UTILIZATION PRACTICES in $ / DOZEN           0
ALL UTILIZATION PRACTICES, TABLE in $ / DOZEN    0
dtype: int64
Program                0
Year                   0
Period                 0
Week Ending         8212
Geo Level              0
State                  0
State ANSI           288
Ag District         8212
Ag District Code    8212
County              8212
County ANSI         8212
Zip Code            8212
Region              8212
watershed_code         0
Watershed           8212
Commodity              0
Data Item              0
Domain                 0
Domain Category        0
Value                  0
CV (%)              8212
dty

In [219]:
#cleaning production_df

#drop unneccessary and empty columns
production_df = production_df.drop(columns=['Program', 'Week Ending', 'Ag District', 'Geo Level', 'Ag District Code', 'County', 'County ANSI', 'Zip Code', 'Region', 'watershed_code', 
                                            'Watershed', 'Commodity', 'Domain', 'Domain Category', 'CV (%)'])

#drop rows with (D) under Value column, represents no data. strip used because there is a space in front of (D).
production_df = production_df[production_df['Value'].str.strip() != '(D)']

#changing 'Value' column dtype to int instead of object for later calculations.
production_df['Value'] = production_df['Value'].str.replace(',', '').astype(int)

#'State ANSI' nulls represent 'OTHER STATES' in 'State' column, dropping these rows
production_df = production_df[production_df['State'] != 'OTHER STATES']

#need to match unit of measurement with prices_df, removing rows not measured by the dozen.
production_df = production_df[~production_df['Data Item'].str.contains('MEASURED IN EGGS')]

#preserving state level data and hatching/table distinction pre merge with prices_df for additional analysis
production_complete_df = production_df.copy()

#aggregate 'Value' for each Year/Period combo so data matches granularity of prices_df (no state by state data)
production_df = production_df.groupby(['Year', 'Period'])['Value'].sum().reset_index()

#rename columns for clarity. MVP of production_df
production_df.columns = ['Year', 'Month', 'Production in Dozens']
print(production_df.head())



   Year Month  Production in Dozens
0  2016   APR             417666800
1  2016   AUG             445508200
2  2016   DEC             465908700
3  2016   FEB             392116900
4  2016   JAN             406550200


In [220]:
#clean prices_df

#retain a complete copy
prices_complete_df =  prices_df

#drop unnecessary columns
prices_df = prices_df.drop(columns=['LOCATION', 'STATE ANSI', 'ASD CODE', 'COUNTY ANSI', 'COMMODITY', 'ALL UTILIZATION PRACTICES, TABLE in $ / DOZEN'])

#rename columns for clarity
prices_df.columns = ['Year', 'Month', 'Price($) per Dozen']
print(prices_df.head())

   Year Month  Price($) per Dozen
0  2016   JAN               1.180
1  2016   FEB               1.080
2  2016   MAR               0.970
3  2016   APR               0.680
4  2016   MAY               0.552


In [221]:
#sqlite database creation

#create connection
conn = sqlite3.connect('egg_data.db')
cursor = conn.cursor()

#create production table
cursor.execute('''
CREATE TABLE IF NOT EXISTS production (
    Year INTEGER,
    Month TEXT,
    Production_in_Dozens REAL,
    PRIMARY KEY (Year, Month)
)
''')

#create prices table
cursor.execute('''
CREATE TABLE IF NOT EXISTS prices (
    Year INTEGER,
    Month TEXT,
    Price_per_Dozen INTEGER,
    PRIMARY KEY (Year, Month)
)
''')

#insert data into tables
production_df.to_sql("production", conn, if_exists="replace", index=False)
prices_df.to_sql("prices", conn, if_exists="replace", index=False)

#verify tables
print(pd.read_sql_query("SELECT * FROM production", conn))
print(pd.read_sql_query("SELECT * FROM prices", conn))

    Year Month  Production in Dozens
0   2016   APR             417666800
1   2016   AUG             445508200
2   2016   DEC             465908700
3   2016   FEB             392116900
4   2016   JAN             406550200
..   ...   ...                   ...
67  2021   MAR             458258800
68  2021   MAY             454082900
69  2021   NOV             452241600
70  2021   OCT             464249700
71  2021   SEP             444491500

[72 rows x 3 columns]
    Year Month  Price($) per Dozen
0   2016   JAN               1.180
1   2016   FEB               1.080
2   2016   MAR               0.970
3   2016   APR               0.680
4   2016   MAY               0.552
..   ...   ...                 ...
67  2021   AUG               1.070
68  2021   SEP               1.030
69  2021   OCT               0.895
70  2021   NOV               1.090
71  2021   DEC               1.360

[72 rows x 3 columns]


In [222]:
#join tables
SELECT
    production.Production_in_Dozens,
    prices.Price_per_Dozen
FROM
    production
INNER JOIN
    prices
ON
    production.Month = prices.Month AND production.Year = prices.Year;

IndentationError: unexpected indent (418872180.py, line 3)